# Vector distance (`cosine`) views

In [ ]:
import numpy as np
import pandas as pd
from ipywidgets import interact, interact_manual, BoundedFloatText, Combobox, Dropdown

In [ ]:
md_df = pd.read_csv('./data/microorganisms_disease_adjacency.csv', index_col=[0])
mf_df = pd.read_csv('./data/microorganisms_food_adjacency.csv', index_col=[0])
df_df = pd.read_csv('./data/disease_food_adjacency.csv', index_col=[0])

In [ ]:
dists = [md_df.values, mf_df.values, df_df.values]

In [ ]:
dataset = [("Microorganisms Vs Disease", 0), ("Microorganisms Vs Food", 1), ("Disease Vs Food", 2)]

def choose_data(data):
    if data == 0:
        return md_df, dists[0], "Microorganisms", "Disease"
    elif data == 1:
        return mf_df, dists[1], "Microorganisms", "Food"
    else:
        return df_df, dists[2], "Disease", "Food"

## Histogram View

In [ ]:
import plotly.express as px

In [ ]:
@interact(data=Dropdown(options=dataset, value=0, description="Dataset: "))
def hist_data(data):
    df, _, row, column = choose_data(data)
    @interact_manual(sim=BoundedFloatText(value=0.5, min=df.min().min(), max=df.max().max(), step=0.01, description="Distance: "),
                    row=Combobox(options=list(df.index) + [''], ensure_option=True, description=f"{row}: "),
                    column=Combobox(options=list(df.columns) + [''], ensure_option=True, description=f"{column}: "))
    def histogram(sim=0.5, row=None, column=None):
        ddf = df[df < sim]
        ddf.dropna(axis=0, how='all', inplace=True)
        ddf.dropna(axis=1, how='all', inplace=True)
        if row:
            if row not in ddf.index:
                print(f"`{row}` is not present in the filtered DF")
                return
            ddf = ddf.loc[[row]]
        if column:
            if column not in ddf.columns:
                print(f"`{column}` is not present in the filtered DF")
                return
            ddf = ddf[[column]]
        return px.imshow(ddf, aspect="auto")

## Network View

In [ ]:
from itertools import chain

import networkx as nx
from scipy import sparse
from pyvis.network import Network

In [ ]:
@interact(data=Dropdown(options=dataset, value=0, description="Dataset: "))
def network(data):
    df, dist, row, column = choose_data(data)
    node_info = {}
    name_id_map = {}
    for idx, name in enumerate(chain(df.index, df.columns)):
        node_info[idx] = {'label': name, 'title': name, 'weight': 10, 'group': int(idx < len(df.index))}
        name_id_map[name] = idx

    def create_network(g: nx.MultiGraph):
        nodes, edges = [], []
        for x, y in g.edges:
            x = int(x)
            y = int(y)
            if x > y:
                x, y = y, x
            y_c = int(y - len(df.index))
            nodes += [(x, node_info[x]), (y, node_info[y])]
            edges.append((x, y, {'value': 1 - dist[x, y_c], 'title': dist[x, y_c]}))
        graph = nx.MultiGraph()
        graph.add_nodes_from(nodes)
        graph.add_edges_from(edges)
        return graph

    @interact_manual(d=BoundedFloatText(value=0.3, min=df.min().min(), max=df.max().max(), step=0.01, description="Distance: "), 
                     view=Dropdown(options=["Neighbors", "Connected Components"], value="Neighbors", description="View: "),
                     row_or_col=Combobox(value='', options=list(name_id_map.keys()) + [''], ensure_option=True, description=f"Filter: "))
    def plot_network(d, view="Neighbors", row_or_col=None):
        nt = Network(width='900px', notebook=True)
        g = nx.bipartite.from_biadjacency_matrix(sparse.csr_matrix(dist <= d))
        if row_or_col:
            idx = name_id_map[row_or_col]
            if not g[idx]:
                print(f"`{row_or_col}` is not present in the filtered Graph")
                return
            g = g.subgraph({idx, *nx.neighbors(g, idx)} if view == "Neighbors" else nx.node_connected_component(g, idx)).copy()
        nt.from_nx(create_network(g))
        return nt.show(f'{row}_{column}_pyvis.html')

## Triangular Network between `Microorganisms`, `Disease` & `Food`

In [ ]:
node_info = {}
name_id_map = {}
boundary = [0, len(md_df.index), len(md_df.index) + len(md_df.columns)]
for idx, name in enumerate(chain(md_df.index, md_df.columns, df_df.columns)):
    node_info[idx] = {'label': name, 'title': name, 'weight': 10, 'group': 0 if idx < boundary[1] else 1 if idx < boundary[2] else 2}
    name_id_map[name] = idx

In [ ]:
ELEVATION = {
    0: (0, boundary[1]),
    1: (0, boundary[2]),
    2: (boundary[1], boundary[2])
}

def prepare_combined_graph(dist):
    graph = nx.MultiGraph()
    nodes, edges = [], []
    for k, gdists in enumerate(dists):
        for x, y in zip(*np.where(gdists <= dist)):
            x, y = map(int, (x, y))
            dist = gdists[x, y]
            x += ELEVATION[k][0]
            y += ELEVATION[k][1]
            nodes += [(x, node_info[x]), (y, node_info[y])]
            edges.append((x, y, {'value': 1 - dist, 'title': dist}))
    graph.add_nodes_from(nodes)
    graph.add_edges_from(edges)
    return graph

In [ ]:
@interact_manual(dist=BoundedFloatText(value=0.3, min=0.0, max=2.0, step=0.01, description="Distance: "),
                row_or_col=Combobox(value='', options=list(name_id_map.keys()) + [''], ensure_option=True, description=f"Filter: "))
def plot_network(dist, row_or_col=None):
    nt = Network(width='900px', notebook=True)
    g = prepare_combined_graph(dist)
    if row_or_col:
        idx = name_id_map[row_or_col]
        if idx not in g.nodes:
            print(f"`{row_or_col}` is not present in the filtered Graph")
            return
        for c in nx.connected_components(g):
            if idx in c:
                g = g.subgraph(c).copy()
                break
    nt.from_nx(g)
    return nt.show(f'triang_pyvis.html')